In [9]:
import pygame
import numpy as np
from game import GameEnvironment
from model import Q_network, get_input
import torch
import sys
import cv2

gridsize = 15 # 13
framerate = 10
block_size = 20
snake_name = 'Snake_game'

model = Q_network(10, 20, 5)
## model.load_state_dict(torch.load('./dir_chk_lr0.0001/Snake_30000'))
model.load_state_dict(torch.load('./check_points_len/Snake_30000'))

board = GameEnvironment(gridsize, 0, 0, 1)
windowwidth = gridsize*block_size*2
windowheight = gridsize*block_size

pygame.init()
win = pygame.display.set_mode((windowwidth, windowheight))
pygame.display.set_caption("Snake")
font = pygame.font.SysFont("Comic Sans MS", 14)
clock = pygame.time.Clock()

def drawboard(snake, apple):
    win.fill((0,0,0))
    for pos in snake.prevpos:
        pygame.draw.rect(win, (255,255,0), (pos[0]*block_size, pos[1]*block_size, block_size, block_size))
    pygame.draw.rect(win, (255, 0, 0), (apple.pos[0]*block_size, apple.pos[1]*block_size, block_size, block_size))
VIDEO = [] # string buffer
runGame = True

prev_len_of_snake = 0
score = 0
while runGame:
    clock.tick(framerate)
    # take actions and draw the game
    state_0 = get_input(board.snake, board.apple)
    state = model(state_0)
    
    action = torch.argmax(state)
    
    reward, done, len_of_snake = board.update_boardstate(action)
    drawboard(board.snake, board.apple)
    score += reward 
    # text
    lensnaketext     = font.render('          LEN OF SNAKE: ' + str(len_of_snake), True, (255, 255, 255))
    rewardtext       = font.render('          SCORE: ' + str(int(score)), True, (255, 255, 255))
    prevlensnaketext = font.render('          LEN OF PREVIOUS SNAKE: ' + str(prev_len_of_snake), False, (255, 255, 255))
    
    win.blit(lensnaketext, (windowwidth//2, 40))
    win.blit(rewardtext, (windowwidth//2, 80))
    win.blit(prevlensnaketext, (windowwidth//2, 120))
    
        
    VIDEO.append(pygame.image.tostring(win, 'RGB', False))
    # handeling events
    for event in pygame.event.get():
        if event.type==pygame.KEYDOWN and event.key == pygame.K_ESCAPE:   
            runGame = False
    #update the game
    pygame.display.update()
    
    if board.game_over == True:
        prev_len_of_snake = len_of_snake
        score = 0
        board.resetgame()
# save a video 
fourcc = cv2.VideoWriter_fourcc(*'MPV4')
output_name = 'output_' + snake_name + '.mp4'
video_mp4 = cv2.VideoWriter(output_name,fourcc, 10.0, (windowwidth,windowheight))        
    
for image in VIDEO:
    
    image = np.frombuffer(image, np.uint8).reshape(windowheight, windowwidth, 3)
    image = cv2.cvtColor(image, cv2.COLOR_RGBA2BGR)
    video_mp4.write(image)

cv2.destroyAllWindows()
video_mp4.release()
        

pygame.quit()            